*Follow the notebook*

https://www.kaggle.com/code/zulqarnainali/explained-detect-ai-generated-text-resourced

In [ ]:
!pip install Datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00


In [ ]:
import sys
import gc
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.metrics import roc_auc_score
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer

from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

from datasets import Dataset
from tqdm.auto import tqdm
from transformers import PreTrainedTokenizerFast

from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier

In [ ]:
from datasets import Dataset


In [ ]:
test = pd.read_csv('test_essays.csv')
sub = pd.read_csv('sample_submission.csv')
org_train = pd.read_csv('train_essays.csv')

In [ ]:
train = pd.read_csv("train_v2_drcat_02.csv", sep=',')

In [ ]:
train.head()

,text,label,prompt_name,source,RDizzl3_seven
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False
1,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False
2,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False
3,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False
4,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False


In [ ]:
train = train.drop_duplicates(subset=['text'])

train.reset_index(drop=True, inplace=True)

In [ ]:
LOWERCASE = False
VOCAB_SIZE = 30522

In [ ]:
# Creating Byte-Pair Encoding tokenizer
raw_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))



# Adding normalization and pre_tokenizer
raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()] + [normalizers.Lowercase()] if LOWERCASE else [])
raw_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()

# Adding special tokens and creating trainer instance
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=special_tokens)



# Creating huggingface dataset object
dataset = Dataset.from_pandas(test[['text']])

def train_corp_iter():
    """
    A generator function for iterating over a dataset in chunks.
    """
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]

# Training from iterator REMEMBER it's training on test set...
raw_tokenizer.train_from_iterator(train_corp_iter(), trainer=trainer)

tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)


tokenized_texts_test = []

# Tokenize test set with new tokenizer
for text in tqdm(test['text'].tolist()):
    tokenized_texts_test.append(tokenizer.tokenize(text))


# Tokenize train set
tokenized_texts_train = []

for text in tqdm(train['text'].tolist()):
    tokenized_texts_train.append(tokenizer.tokenize(text))


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/44868 [00:00<?, ?it/s]

In [ ]:
tokenized_texts_test[1][1]


'Ġccc'

In [ ]:
# Define a dummy function for custom tokenization
def dummy(text):
    return text

# Initialize a TF-IDF vectorizer with custom tokenization settings
vectorizer = TfidfVectorizer(
    ngram_range=(3, 5),            # 📏 Specify the range of n-grams (3 to 5)
    lowercase=False,              # 🔠 Keep the original case of the text
    sublinear_tf=True,            # 📈 Apply sublinear term frequency scaling
    analyzer='word',              # 📖 Analyze based on word-level tokens
    tokenizer=dummy,              # 🤖 Use the dummy function for custom tokenization
    preprocessor=dummy,           # 🤖 Use the dummy function for custom preprocessing
    token_pattern=None,           # 🧩 Use the dummy function for token pattern
    strip_accents='unicode'       # 🌐 Strip accents using Unicode normalization
)

# Fit the TF-IDF vectorizer on the tokenized texts of the test data
vectorizer.fit(tokenized_texts_test)

# Getting the vocabulary from the fitted vectorizer
vocab = vectorizer.vocabulary_

print(vocab)

# Reinitialize the TF-IDF vectorizer with the obtained vocabulary
vectorizer = TfidfVectorizer(
    ngram_range=(3, 5),           # 📏 Specify the range of n-grams (3 to 5)
    lowercase=False,              # 🔠 Keep the original case of the text
    sublinear_tf=True,            # 📈 Apply sublinear term frequency scaling
    vocabulary=vocab,             # 📖 Use the obtained vocabulary
    analyzer='word',              # 📖 Analyze based on word-level tokens
    tokenizer=dummy,              # 🤖 Use the dummy function for custom tokenization
    preprocessor=dummy,           # 🤖 Use the dummy function for custom preprocessing
    token_pattern=None,           # 🧩 Use the dummy function for token pattern
    strip_accents='unicode'       # 🌐 Strip accents using Unicode normalization
)

# Transform the training and test data using the reinitialized vectorizer
tf_train = vectorizer.fit_transform(tokenized_texts_train)
tf_test = vectorizer.transform(tokenized_texts_test)

# Clean up memory by deleting the vectorizer and collecting garbage
del vectorizer
gc.collect()

{'ĠAaa Ġbbb Ġccc': 0, 'Ġbbb Ġccc .': 6, 'ĠAaa Ġbbb Ġccc .': 1, 'ĠBbb Ġccc Ġddd': 2, 'Ġccc Ġddd .': 7, 'ĠBbb Ġccc Ġddd .': 3, 'ĠCCC Ġddd Ġeee': 4, 'Ġddd Ġeee .': 8, 'ĠCCC Ġddd Ġeee .': 5}


42

In [ ]:
# Extracting the 'label' column values from the 'train' DataFrame and storing them in y_train
y_train = train['label'].values

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.3 MB/s eta 0:00:00


In [ ]:
# prompt: LGBMClassifier

from lightgbm import LGBMClassifier


In [ ]:
# Define a function to get the ensemble model consisting of various classifiers
def get_model():
    from catboost import CatBoostClassifier
    from sklearn.linear_model import LogisticRegression
    from lightgbm import LGBMClassifier
    # Multinomial Naive Bayes classifier with specified alpha
    clf = MultinomialNB(alpha=0.0225)

    # Stochastic Gradient Descent (SGD) classifier with custom settings
    sgd_model = SGDClassifier(max_iter=9000, tol=1e-4, loss="modified_huber", random_state=6743)

    # LightGBM classifier with hyperparameters defined in dictionary p6
    p6 = {'n_iter': 3000, 'verbose': -1, 'objective': 'cross_entropy', 'metric': 'auc',
          'learning_rate': 0.00581909898961407, 'colsample_bytree': 0.78, 'colsample_bynode': 0.8}
    p6["random_state"] = 6743
    lgb = LGBMClassifier(**p6)

    # CatBoost classifier with specified iterations, learning rate, and subsample
    cat = CatBoostClassifier(iterations=3000, verbose=0, random_seed=6543,
                             learning_rate=0.005599066836106983, subsample=0.35,
                             allow_const_label=True, loss_function='CrossEntropy')

    # Weights for the Voting Classifier, specifying the importance of each base classifier
    weights = [0.1, 0.31, 0.28, 0.67]

    # Create a Voting Classifier with the specified base classifiers and weights
    ensemble = VotingClassifier(estimators=[('mnb', clf),
                                            ('sgd', sgd_model),
                                            ('lgb', lgb),
                                            ('cat', cat)
                                            ],
                                weights=weights, voting='soft', n_jobs=-1)
    return ensemble

# Get the ensemble model
model = get_model()
print(model)

# Check if the number of test samples is less than or equal to 5
if len(test.text.values) <= 5:
    # If true, generate a sample submission file
    sub.to_csv('submission.csv', index=False)
else:
    # If false, fit the ensemble model on the training data
    model.fit(tf_train, y_train)

    # Predict probabilities for the test data
    final_preds = model.predict_proba(tf_test)[:, 1]

    # Add the generated probabilities to the submission DataFrame
    sub['generated'] = final_preds

    # Save the submission file without index column
    sub.to_csv('submission.csv', index=False)
    sub

VotingClassifier(estimators=[('mnb', MultinomialNB(alpha=0.0225)),
                             ('sgd',
                              SGDClassifier(loss='modified_huber',
                                            max_iter=9000, random_state=6743,
                                            tol=0.0001)),
                             ('lgb',
                              LGBMClassifier(colsample_bynode=0.8,
                                             colsample_bytree=0.78,
                                             learning_rate=0.00581909898961407,
                                             metric='auc', n_iter=3000,
                                             objective='cross_entropy',
                                             random_state=6743, verbose=-1)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x7bca40ac8490>)],
                 n_jobs=-1, voting='soft', weights=[0.1, 0.31, 0.28, 0.67])
